1. Setup and imports
Add in the tols necessary for analysis

mne -> EEG processing
numpy -> numerical operations
os -> handle file paths

In [1]:
import mne
import numpy as np
import os
import sys
print(sys.executable)

%matplotlib qt


c:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\.venv-gpu\Scripts\python.exe


2. Configuration
Easy modification of the subject_id / task for analysis without scrolling for code

In [ ]:
subject_id = 1 # which subject to process
task_name = 'imagined_movement' #which specific run

task_run = {
    'imagined_movement' : [4, 8, 12], # Left vs Right Fist IMAGERY
    'actual_movement' : [3, 7 , 11]   # Left vs Right First Actual movement

}
runs_to_load = task_run[task_name]

3. Data Loading & Combination
Use MNE built-in downloader
Download only necessary files for cleaning and analysing

Data is stored in: "C:\Users\524yu\mne_data"

In [3]:
file_paths = mne.datasets.eegbci.load_data(subjects=[subject_id], runs=runs_to_load)

raw_files = [mne.io.read_raw_edf(path, preload=True, stim_channel='auto') for path in file_paths]

raw_combined = mne.concatenate_raws(raw_files)

print("data loaded and combined")
raw_combined.info

Extracting EDF parameters from C:\Users\524yu\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\524yu\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\524yu\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
data loaded and combined


<Info | 8 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
 subject_info: <subject_info | his_id: X, sex: 0, last_name: X>
>

4. Prepocessing portion: Filtering & montage setup
- bandpass filter
-fix  channel names to set up a standard montage (physionet data doesn't have it built in)
-standard montage: 3D location of the electrode
-Important for autamted ICA in stage 2 for other patient data

In [4]:
raw_processed = raw_combined.copy()

mapping = {ch_name: ch_name.strip('.') for ch_name in raw_processed.ch_names}
raw_processed.rename_channels(mapping)

raw_processed.set_montage('standard_1005', match_case=False)
raw_processed.filter(l_freq=1., h_freq=40.)
print("filtering & montage setup complete")

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

filtering & montage setup complete


5. Automated Artifact Removal (ICA)

In [5]:
from mne.preprocessing import ICA

# Your existing ICA setup code
ica = ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw_processed)

# THE FIX: Tell the function which channels to use as the EOG reference
eog_indices, eog_scores = ica.find_bads_eog(
    raw_processed,
    ch_name=['Fp1', 'Fp2'] # Use the frontal pole channels closest to the eyes
)

print(f"✅ Automatically detected eye-blink component(s): {eog_indices}")

# The rest of your code is correct
ica.exclude = eog_indices
raw_cleaned = raw_processed.copy()
ica.apply(raw_cleaned)

print("✅ ICA cleaning completed.")

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 1.8s.
Using EOG channels: Fp1, Fp2
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 1600 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 H

6. Epoching; segmenting into trials

Locate T1 (left fist), T2 (right fist) markers

Cut a 4 second window for each marker to create the labelled trials

In [6]:
print(raw_cleaned.ch_names)

['Fc5', 'Fc3', 'Fc1', 'Fcz', 'Fc2', 'Fc4', 'Fc6', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'Cp5', 'Cp3', 'Cp1', 'Cpz', 'Cp2', 'Cp4', 'Cp6', 'Fp1', 'Fpz', 'Fp2', 'Af7', 'Af3', 'Afz', 'Af4', 'Af8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'Ft7', 'Ft8', 'T7', 'T8', 'T9', 'T10', 'Tp7', 'Tp8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'Po7', 'Po3', 'Poz', 'Po4', 'Po8', 'O1', 'Oz', 'O2', 'Iz']


In [ ]:
# --- This is the corrected code for your epoching cell ---

print("--- Creating epochs object (without preloading)... ---")

events, event_id_from_annot = mne.events_from_annotations(raw_cleaned)
event_id = {'left_fist': event_id_from_annot['T1'], 'right_fist': event_id_from_annot['T2']}

# THE FIX: Change preload to False.
epochs = mne.Epochs(
    raw_cleaned,
    events,
    event_id=event_id,
    tmin=-0.5,
    tmax=4.0,
    preload=False, # This creates the object instantly without loading data
    baseline=(-0.5, 0),
    reject=dict(eeg=200e-6)
)

print("✅ Epochs object created. Now loading data into memory...")

# Now, explicitly load the data. This will show progress.
epochs.load_data()

print("✅ Epoching complete. Final trials:")
print(epochs)

# This plot should now work
print("\n--- Drop Log ---")
epochs.plot_drop_log()

7. Final Data Extraction for Machine Learning

Extract data from MNE epoch objects -> numpy arrays

X: Data
y: labels

In [7]:
# Get the EEG data as a NumPy array: (n_trials, n_channels, n_time_points)
X = epochs.get_data()

# Get the corresponding labels as a NumPy array: (n_trials,)
y = epochs.events[:, -1] # The last column of the events array contains the label ID

print(f"Data extracted for ML")
print(f"Shape of X (data): {X.shape}")
print(f"Shape of y (labels): {y.shape}")

NameError: name 'epochs' is not defined